Используя код лабораторной работы 2, собрать 50 новостей с сайта в DataFrame. - он ниже
Произвести предобработку всех данных (удалить знаки препинания, составить частотный словарь, закодировать в Bag of Words).
Дополнительно: провести лемматизацию и закодировать с помощью любого предобученного эмбеддинга.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re

In [2]:
def parse_ria_news(date_from=None, max_news=50):
    """
    Парсит до max_news новостей с РИА Новости (раздел world) с пагинацией.
    Возвращает DataFrame c колонками ['date', 'title', 'text', 'link'].
    """
    import requests
    from bs4 import BeautifulSoup
    from datetime import datetime
    import re

    section_url = "https://ria.ru/world/"
    base_url = "https://ria.ru"
    headers = {"User-Agent": "Mozilla/5.0"}
    news_list = []
    page = 1

    while len(news_list) < max_news:
        url = section_url if page == 1 else f"{section_url}?page={page}"
        resp = requests.get(url, headers=headers)
        if resp.status_code != 200:
            break
        soup = BeautifulSoup(resp.text, "html.parser")
        news_cards = soup.find_all("div", class_="list-item__content")
        if not news_cards:
            break

        for card in news_cards:
            if len(news_list) >= max_news:
                break
            title_tag = card.find("a", class_="list-item__title")
            if not title_tag:
                continue
            title = title_tag.text.strip()
            link = title_tag["href"]
            if link.startswith("/"):
                link = base_url + link

            date_tag = card.find("span", class_="list-item__date")
            if date_tag:
                date_text = date_tag.text.strip()
                try:
                    news_date = datetime.strptime(date_text, "%d.%m.%Y %H:%M")
                except Exception:
                    news_date = None
            else:
                news_date = None

            if date_from:
                date_from_dt = datetime.strptime(date_from, "%Y-%m-%d")
                if news_date and news_date < date_from_dt:
                    continue

            news_resp = requests.get(link, headers=headers)
            if news_resp.status_code != 200:
                continue
            news_soup = BeautifulSoup(news_resp.text, "html.parser")

            date_match = re.search(r"/(\d{8})/", link)
            if date_match:
                news_date = datetime.strptime(date_match.group(1), "%Y%m%d")
            text_tag = news_soup.find("div", class_="article__body")
            text = text_tag.get_text(separator="\n").strip() if text_tag else ""

            news_list.append(
                {"date": news_date, "title": title, "text": text, "link": link}
            )

        page += 1

    df = pd.DataFrame(news_list)
    return df

In [3]:
df_news = parse_ria_news(max_news=50)

In [4]:
df_news.sample(5)

,date,title,text,link
43,2025-12-05,Путин заявил об устойчивом темпе роста товароо...,РИА Новости ТОВАРООБОРОТ РФ И ИНДИИ ПРОДОЛЖАЕТ...,https://ria.ru/20251205/indiya-2060071902.html
23,2025-12-05,Россия направит представительство на саммит по...,РИА Новости РОССИЯ НАПРАВИТ ПРЕДСТАВИТЕЛЬНУЮ Д...,https://ria.ru/20251205/indiya-2060073696.html
13,2025-12-05,Путин и Моди приехали на Российско-индийский б...,"НЬЮ-ДЕЛИ, 5 дек - РИА Новости.\n Президент Рос...",https://ria.ru/20251205/forum-2060066194.html
28,2025-12-05,В НАТО назвали переговоры по Украине самыми ва...,"МОСКВА, 5 дек - РИА Новости.\n Переговоры об у...",https://ria.ru/20251205/nato-2060068775.html
21,2025-12-05,США ожидают до десяти миллионов иностранных го...,"МОСКВА, 5 дек - РИА Новости.\n Власти Соединён...",https://ria.ru/20251205/ssha-2060076173.html


In [5]:
import numpy as np
import string
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
ru_stop = set(stopwords.words("russian"))


def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    txt = text.lower()
    txt = re.sub(r"[^\w\s]", " ", txt)  # убрать пунктуацию
    txt = re.sub(r"\d+", " ", txt)  # убрать числа
    txt = re.sub(r"\s+", " ", txt).strip()
    return txt


df_news = df_news.copy()

df_news["text_clean"] = df_news["text"].apply(clean_text)
df_news["title_clean"] = df_news["title"].apply(clean_text)

df_news["text_len"] = df_news["text_clean"].str.split().str.len()
print("Всего новостей:", len(df_news))
print("Пустых после заполнения:", int((df_news["text_clean"] == "").sum()))
df_news[["date", "title_clean", "text_clean"]].head()

Всего новостей: 50
Пустых после заполнения: 0


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/arsknz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,date,title_clean,text_clean
0,2025-12-05,европа склоняется к тому что киеву придется по...,москва дек риа новости страны европы все больш...
1,2025-12-05,путин раскрыл что нужно сделать в отношениях м...,нью дели дек риа новости президент россии влад...
2,2025-12-05,сша ожидают до десяти миллионов иностранных го...,москва дек риа новости власти соединённых штат...
3,2025-12-05,путин отметил важность заключения торгового до...,нью дели дек риа новости скорейшее заключение ...
4,2025-12-05,россия направит представительство на саммит по...,риа новости россия направит представительную д...


In [6]:
from itertools import chain

tokens = list(chain.from_iterable(df_news["text_clean"].str.split()))
tokens = [t for t in tokens if t and t not in ru_stop]
freq = Counter(tokens)
freq_df = pd.DataFrame(freq.most_common(30), columns=["token", "freq"])
freq_df.head(20)

,token,freq
0,путин,84
1,индии,69
2,россии,54
3,риа,52
4,новости,52
5,россия,51
6,дели,51
7,президент,49
8,нью,47
9,моди,47


In [7]:
vectorizer = CountVectorizer(
    max_features=5000, ngram_range=(1, 2), min_df=2, stop_words=list(ru_stop)
)
X_bow = vectorizer.fit_transform(df_news["text_clean"])
bow_sample = pd.DataFrame(
    X_bow.toarray()[:5], columns=vectorizer.get_feature_names_out()
)
print("Размер матрицы BoW:", X_bow.shape)
bow_sample.head()

Размер матрицы BoW: (50, 2906)


,affinity,affinity partners,bharatmandapam,bharatmandapam крупнейшем,ficci,ficci форум,india,india новый,maidan,maidan нью,...,языка крым,языка нарушает,языке,языке году,языковой,языковой вопрос,языковом,языковом вопросе,ясно,ясно осталась
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Лемматизация (spaCy) и эмбеддинги без try/except
import nltk

nltk.download("punkt")
import spacy
from sentence_transformers import SentenceTransformer

nlp = spacy.load("ru_core_news_sm")


def lemmatize_text(text: str) -> str:
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])


df_news["text_lemma"] = df_news["text_clean"].apply(lemmatize_text)
df_news.sample(5)

[nltk_data] Downloading package punkt to /Users/arsknz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Users/arsknz/miniforge3/envs/main_nn/lib/python3.10/site-packages/spacy/util.py:922: UserWarning: [W095] Model 'ru_core_news_sm' (3.7.0) was trained with spaCy v3.7.0 and may not be 100% compatible with the current version (3.8.7). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/Users/arsknz/miniforge3/envs/main_nn/lib/python3.10/site-packages/spacy/util.py:922: UserWarning: [W095] Model 'ru_core_news_sm' (3.7.0) was trained with spaCy v3.7.0 and may not be 100% compatible with the current version (3.8.7). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run

,date,title,text,link,text_clean,title_clean,text_len,text_lemma
42,2025-12-05,Россия направит представительство на саммит по...,РИА Новости РОССИЯ НАПРАВИТ ПРЕДСТАВИТЕЛЬНУЮ Д...,https://ria.ru/20251205/indiya-2060073696.html,риа новости россия направит представительную д...,россия направит представительство на саммит по...,18,риа новость россия направить представительный ...
37,2025-12-05,Россия и Индия предпринимают меры для заключен...,"НЬЮ-ДЕЛИ, 5 дек - РИА Новости.\n Премьер-минис...",https://ria.ru/20251205/torgovlya-2060063168.html,нью дели дек риа новости премьер министр индии...,россия и индия предпринимают меры для заключен...,130,нью дели дек риа новость премьер министр индия...
19,2025-12-05,"Европа склоняется к тому, что Киеву придется п...","МОСКВА, 5 дек - РИА Новости.\n Страны Европы в...",https://ria.ru/20251205/evropa-2060078751.html,москва дек риа новости страны европы все больш...,европа склоняется к тому что киеву придется по...,435,москва дек риа новость страна европа все больш...
25,2025-12-05,"""Корона бы не упала"". Глава МИД Польши резко н...","МОСКВА, 5 дек — РИА Новости. \nВладимиру Зелен...",https://ria.ru/20251205/sikorskiy-2060071053.html,москва дек риа новости владимиру зеленскому ст...,корона бы не упала глава мид польши резко набр...,134,москва дек риа новость владимир зеленскому сто...
46,2025-12-05,Под руководством Моди Индия проводит независим...,"НЬЮ-ДЕЛИ, 5 дек - РИА Новости.\n Президент Рос...",https://ria.ru/20251205/putin-2060070656.html,нью дели дек риа новости президент россии влад...,под руководством моди индия проводит независим...,56,нью дели дек риа новость президент россия влад...


In [14]:
model = SentenceTransformer("google/embeddinggemma-300m")

modules.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/997 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/18.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/58.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/312 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

3_Dense/model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

In [10]:
model.encode("Hello world")

array([-0.0011777 ,  0.00793466, -0.01161492, ...,  0.00183558,
       -0.00926816,  0.00920283], shape=(1024,), dtype=float32)

In [11]:
embeddings = model.encode(df_news["text_lemma"].tolist(), show_progress_bar=True)
embed_preview = pd.DataFrame(embeddings[:5, :5])
print("размер:", embeddings.shape)
embed_preview.head()

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 5.62 GiB, other allocations: 14.78 MiB, max allowed: 9.07 GiB). Tried to allocate 3.52 GiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).